In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install rasterio

  Using cached rasterio-1.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 44.3 MB/s eta 0:00:00


In [5]:
import os
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from glob import glob

In [ ]:


# 📁 Ruta a la carpeta de escenas
data_folder = "/content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/Sentinel2_Exports"

# 🔍 Encuentra todas las fechas
files = glob(os.path.join(data_folder, "*.tif"))
dates = sorted(list(set([os.path.basename(f).split("_")[2] for f in files])))

# 📤 Carpeta de salida
output_folder = os.path.join(data_folder, "processed")
os.makedirs(output_folder, exist_ok=True)

def merge_tiles(file_pattern):
    """Merge mosaics G + F for a single band type."""
    matching_files = sorted(glob(file_pattern))
    srcs = [rasterio.open(fp) for fp in matching_files]
    merged, out_trans = merge(srcs)
    out_meta = srcs[0].meta.copy()
    out_meta.update({
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": out_trans
    })
    for src in srcs:
        src.close()
    return merged, out_meta

def resample_to_10m(src_array, src_meta, target_shape, target_transform, method=Resampling.bilinear):
    """Resample input array to match 10m target resolution."""
    dst_array = np.empty(shape=target_shape, dtype=src_meta['dtype'])
    reproject(
        source=src_array,
        destination=dst_array,
        src_transform=src_meta['transform'],
        src_crs=src_meta['crs'],
        dst_transform=target_transform,
        dst_crs=src_meta['crs'],
        resampling=method
    )
    return dst_array

# 🧠 Proceso por cada fecha
for date in dates:
    print(f"Procesando fecha: {date}")

    # Merge por tipo y mosaico (G/F)
    merged_data = {}
    for res in ["10m", "20m", "60m", "SCL"]:
        pattern = os.path.join(data_folder, f"S2_{res}_{date}_*.tif")
        arr, meta = merge_tiles(pattern)
        merged_data[res] = {"array": arr, "meta": meta}

    # Usamos como referencia la resolución 10m
    base_meta = merged_data["10m"]["meta"]
    base_shape = merged_data["10m"]["array"].shape
    base_transform = base_meta["transform"]

    # Resamplea bandas 20m y 60m a 10m
    bands_resampled = [merged_data["10m"]["array"]]  # ya está a 10m
    for res in ["20m", "60m"]:
        arr = merged_data[res]["array"]
        meta = merged_data[res]["meta"]
        for band in arr:
            band_resampled = resample_to_10m(
                band,
                meta,
                target_shape=base_shape[1:],  # (height, width)
                target_transform=base_transform
            )
            bands_resampled.append(band_resampled[np.newaxis, :, :])  # agregar como banda nueva

    # Agrega banda SCL sin reescalar
        # === SCL ===
    scl_array = merged_data["SCL"]["array"]
    scl_meta = merged_data["SCL"]["meta"]
    scl_resampled = resample_to_10m(
        scl_array[0],  # Solo una banda
        scl_meta,
        target_shape=base_shape[1:],
        target_transform=base_transform,
        method=Resampling.nearest  # no interpolar clasificaciones
    )
    bands_resampled.append(scl_resampled[np.newaxis, :, :])  # una banda extra

    # Stack final
    full_stack = np.concatenate(bands_resampled, axis=0)
    final_meta = base_meta.copy()
    final_meta.update({
        "count": full_stack.shape[0],
        "dtype": full_stack.dtype,
        "driver": "GTiff"
    })

    # Nombres para las bandas (ejemplo: B1_10m, B1_20m...)
    #band_names = (
    #    [f"B{i+1}_10m" for i in range(merged_data["10m"]["array"].shape[0])] +
    #    [f"B{i+1}_20m" for i in range(merged_data["20m"]["array"].shape[0])] +
    #    [f"B{i+1}_60m" for i in range(merged_data["60m"]["array"].shape[0])] +
    #    ["SCL"]
    #)
    band_names = (
     ['B2', 'B3', 'B4', 'B8'] +
     ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'] +
     ['B1', 'B9'] +
     ['SCL']
    )
    # Guardar archivo
    out_path = os.path.join(output_folder, f"S2_stack_{date}_10m.tif")
    with rasterio.open(out_path, "w", **final_meta) as dst:
        for i in range(full_stack.shape[0]):
            dst.write(full_stack[i, :, :], i + 1)
            dst.set_band_description(i + 1, band_names[i])

    print(f"✔️ Raster guardado: {out_path}")


Procesando fecha: 2024-03-08


In [8]:
import os
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from glob import glob

# 📁 Ruta de trabajo
data_folder = "/content/drive/MyDrive/Programming/Colab Notebooks/Fray Jorge LULC/FrayJorge/input/S2/test"
output_folder = os.path.join(data_folder, "processed1")
os.makedirs(output_folder, exist_ok=True)

# Bandas por resolución
bands_10m = ['B2', 'B3', 'B4', 'B8']
bands_20m = ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12']
bands_60m = ['B1', 'B9']
bands_scl = ['SCL']

# Extrae fechas
files = glob(os.path.join(data_folder, "*.tif"))
dates = sorted(list(set([os.path.basename(f).split("_")[2] for f in files])))

def merge_tiles(file_pattern):
    matching_files = sorted(glob(file_pattern))
    srcs = [rasterio.open(fp) for fp in matching_files]
    merged, out_trans = merge(srcs)
    out_meta = srcs[0].meta.copy()
    out_meta.update({
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": out_trans
    })
    for src in srcs:
        src.close()
    return merged, out_meta

def resample_band(band_array, src_meta, target_shape, target_transform, method):
    dst_array = np.empty(shape=target_shape, dtype=band_array.dtype)
    reproject(
        source=band_array,
        destination=dst_array,
        src_transform=src_meta['transform'],
        src_crs=src_meta['crs'],
        dst_transform=target_transform,
        dst_crs=src_meta['crs'],
        resampling=method
    )
    return dst_array

# Procesamiento por fecha
for date in dates:
    print(f"🕓 Procesando fecha: {date}")
    stack_bands = []
    band_names = []

    # === 10m ===
    for b in bands_10m:
        pattern = os.path.join(data_folder, f"*10m_{date}_*_{b}*.tif")
        arr, meta = merge_tiles(pattern)
        stack_bands.append(arr[0])
        band_names.append(b)

    ref_shape = stack_bands[0].shape
    ref_transform = meta['transform']
    ref_crs = meta['crs']

    # === 20m ===
    for b in bands_20m:
        pattern = os.path.join(data_folder, f"*20m_{date}_*_{b}*.tif")
        arr, meta_20 = merge_tiles(pattern)
        band_10m = resample_band(arr[0], meta_20, ref_shape, ref_transform, Resampling.bilinear)
        stack_bands.append(band_10m)
        band_names.append(b)

    # === 60m ===
    for b in bands_60m:
        pattern = os.path.join(data_folder, f"*60m_{date}_*_{b}*.tif")
        arr, meta_60 = merge_tiles(pattern)
        band_10m = resample_band(arr[0], meta_60, ref_shape, ref_transform, Resampling.bilinear)
        stack_bands.append(band_10m)
        band_names.append(b)

    # === SCL ===
    pattern = os.path.join(data_folder, f"*SCL_{date}_*.tif")
    arr, meta_scl = merge_tiles(pattern)
    scl_10m = resample_band(arr[0], meta_scl, ref_shape, ref_transform, Resampling.nearest)
    stack_bands.append(scl_10m)
    band_names.append('SCL')

    # === Guardar raster ===
    full_stack = np.stack(stack_bands)
    out_meta = meta.copy()
    out_meta.update({
        "count": full_stack.shape[0],
        "dtype": full_stack.dtype,
        "driver": "GTiff"
    })

    out_path = os.path.join(output_folder, f"S2_stack_{date}_10m.tif")
    with rasterio.open(out_path, "w", **out_meta) as dst:
        for i in range(full_stack.shape[0]):
            dst.write(full_stack[i, :, :], i + 1)
            dst.set_band_description(i + 1, band_names[i])

    print(f"✅ Guardado: {out_path}")


🕓 Procesando fecha: 2024-03-08


IndexError: list index out of range